# Using Landsat ARD

This notebook demonstrates how to retrieve and use Landsat ARD. _It is a work in progress!_

## The Python LCMAP Client

To use the client, import the `lcmap_client` module and instantiate a client object. The client uses a configuration file in your home directory `~/.usgs/lcmap.ini` to locate the LCMAP REST API.

In [1]:
from lcmap_client import Client
client = Client();

## Finding Tiles

Finding tiles requires a universal band identifier (UBID), a point, and a time interval. This will return a pair of values: a tile specification and a list of tiles.

* A UBID is a combination of mission, sensor, and band short name.
* X and Y values are in terms of projection system coordinates.
* Times are specified as ISO-8601 dates.

In the future, changes will be made to support a variety parameter types to make usage more convenient.

In [3]:
ubid = "LANDSAT_5/TM/sr_band1"
x, y = -2096625, 3095025
t1, t2 = '1985-01-01', '1985-02-01'
spec, tiles = client.data.surface_reflectance.tiles(ubid, x, y, t1, t2)

Let's examine the first tile and see how to retrieve a pixel value.

In [4]:
tile = tiles[0]

Each tile has a data property that can be used to access and, masked, scaled, and shaped set of values.

In [5]:
tile.data[0,0]

0.25019999367941637

You may also get data values in terms of the projection system coordinates. This is much more convenient than calculating the raster grid projection coordinate containing a point in the projection system on your own.

In [6]:
tiles[0][-2096625, 3095025]

0.25019999367941637

Notice that the point you specify can be "off" the implicit grid, here we move east/south by two meters. These values are _snapped_ to the nearest point.

In [7]:
tiles[0][-2096625+2, 3095025-2]

0.25019999367941637

If you needed, you can also get the acquisition date and scene ID of each tile.

In [9]:
[(t.acquired, t.source) for t in tiles]

[('1985-01-02T06:00:00Z', 'LT50470271985002XXX03'),
 ('1985-01-09T06:00:00Z', 'LT50480271985009PAC00'),
 ('1985-01-18T06:00:00Z', 'LT50470271985018XXX03'),
 ('1985-01-25T06:00:00Z', 'LT50480271985025PAC00')]

## Creating an Image

The client currently provides very basic functionality. If you want to render a larger image you can define a simple function to get tiles for a wider area and multiple bands. Note, this function omits the returned specs because we don't have a use for the information they contain.

In [14]:
def get_tiles(bands, x1, y1, x2, y2, t1, t2):
    xs = range(x1, x2,  256*30)
    ys = range(y1, y2, -256*30)
    results = []
    for band in bands:
        for x in xs:
            for y in ys:
                _, tiles = client.data.surface_reflectance.tiles(band, x, y, t1, t2)
                results.append(tiles)
    return results

Use the `get_tiles` function to retrieve multiple bands of data over a wide area. The returned value is a list-of-lists of tiles.

In [15]:
bands = ["LANDSAT_5/TM/sr_band3", "LANDSAT_5/TM/sr_band2", "LANDSAT_5/TM/sr_band1"]
x1, y1 = -2096625, 3095025
x2, y2 = -2096625+(256*30)+1, 3095025-(256*30)-1
t1, t2 = '1985-01-01', '1985-02-01'
tiles = get_tiles(bands, x1, y1, x2, y2, t1, t2)

### Melding Tile Data

Once we have a list-of-lists, we can meld them into three "layers" of data by defining another function. We use `numpy` and `pandas` to create this new data structure.

In [19]:
def meld(ts, ix):
    return [tiles[ix] for tiles in ts]

In [21]:
[(t.x,t.y,t.ubid) for t in meld(tiles, 0)]

[(-2096625, 3095025, 'LANDSAT_5/TM/sr_band3'),
 (-2096625, 3087345, 'LANDSAT_5/TM/sr_band3'),
 (-2088945, 3095025, 'LANDSAT_5/TM/sr_band3'),
 (-2088945, 3087345, 'LANDSAT_5/TM/sr_band3'),
 (-2096625, 3095025, 'LANDSAT_5/TM/sr_band2'),
 (-2096625, 3087345, 'LANDSAT_5/TM/sr_band2'),
 (-2088945, 3095025, 'LANDSAT_5/TM/sr_band2'),
 (-2088945, 3087345, 'LANDSAT_5/TM/sr_band2'),
 (-2096625, 3095025, 'LANDSAT_5/TM/sr_band1'),
 (-2096625, 3087345, 'LANDSAT_5/TM/sr_band1'),
 (-2088945, 3095025, 'LANDSAT_5/TM/sr_band1'),
 (-2088945, 3087345, 'LANDSAT_5/TM/sr_band1')]

In [26]:
rgb = meld(tiles, 0)

### Rendering an Image

In order to render an image, we use `matplotlob` and `skimage` -- the latter has a function for color correcting images.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import matplotlib.cm as cm
import skimage.exposure as ex
%matplotlib inline

In [ ]:
n = mc.Normalize(0.0, 1.0, clip=True)
rgb_ah = ex.equalize_adapthist(n(rgb), clip_limit=0.05)
fig = plt.figure(figsize=(8,8))
plt.axis('off')
plt.imshow(rgb_ah, interpolation='nearest')